In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import os

save_path = "match_result/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
db = sqlite3.connect("../db.sqlite3")
applicants = pd.read_sql_query("SELECT * FROM applicant", db)
matches = pd.read_sql_query("SELECT * FROM match", db)
matches
successed_matches = matches[ (matches["applicant1_status"] == "A") & (matches["applicant2_status"] == "A") ]
successed_matches

In [ ]:
successed_applicants = pd.concat([successed_matches["applicant1_id"], successed_matches["applicant2_id"]])
successed_applicants.shape

In [ ]:
failed_matches = matches[ (matches["applicant1_status"] != "A") | (matches["applicant2_status"] != "A") ]
failed_matches

In [ ]:
no_response_matches = failed_matches[ failed_matches["discarded"] == 0]
no_response_matches

In [ ]:
no_response_applicants = pd.concat([no_response_matches["applicant1_id"], no_response_matches["applicant2_id"]])
no_response_applicants.to_csv( save_path + "no_response_applicants.csv", index=False, header=False)
no_response_applicants.shape

# danger zone

In [ ]:

proceed = input("Are you sure to proceed? (yes/n)")
if proceed == "yes":
    # for every successed applicant, update their "confirmed" to 1 in database
    cursor = db.cursor()

    for applicant_id in successed_applicants:
        cursor.execute("UPDATE applicant SET confirmed = 1 WHERE id = ?", (applicant_id,))

    db.commit()

    # for every no_response match, update their "discarded" to 1 in database and set the "discard_reason" to "匹配确认超时, 系统自动废弃"

    for match_id in no_response_matches["id"]:
        cursor.execute("UPDATE match SET discarded = 1, discard_reason = '匹配确认超时, 系统自动废弃' WHERE id = ?", (match_id,))

    db.commit()
    
else:
    print("Operation cancelled")